In [ ]:
#!/usr/bin/python

# code modified from http://web.stanford.edu/~zlotnick/TextAsData/Web_Scraping_with_Beautiful_Soup.html

from bs4 import BeautifulSoup

docfile = open("nn.html", "r")

docbuff = docfile.readlines()

docfile.close()
## was concerned this would be too long, seems fine tho. Only 1.1M
# docbuff = "nothing here yet"
# while docbuff != None:
# print(docfile.tell())

In [2]:


soup = BeautifulSoup(docbuff[0], "html.parser")


In [9]:
categories = soup.body.find_all(["h1","h2","h3", "h4", "h5", "h6"])

print "categories length", len(categories)
for category in categories:
    if not category.text or len(category.text[1]) < 1:
        print category.contents, "is empty"
        category.decompose()

print "categories now length", len(categories)


# print soup.body.prettify()[5000:10000]
print
for category in categories:
    print category.name[1] + "," + category.text
    
# manually saved categories as csv

# category_strings = [category.text for category in categories]

categories length 99
categories now length 99

1, How to Implement Cutting Edge Neural Networks
1, Basics
2, Libraries
2, Debugging
2, Classification
2, Attention
2, Initialization
2, Activation functions
1, Data collection
2, In-training Dataset Labeling
2, Human-compatibility
1, RL
2, Misc
2, Q-learning
2, Useful for RL
1, AGI Safety/reliable reinforcement learners
1, Unsupervised
2, Automatic labeling
2, Misc Representation Learning
2, Generative modeling
2, Partially supervised training
1, Sparsity
1, Architecture
2, Skip-ahead connections
2, Very Deep/Vanishing Gradients Problem
2, Variable Compute
1, Recurrence
1, Attention
1, Memory
5, application: misc
5, application: language modeling
1, META
2, Optimizers / Learning Algorithms
2, Hyperparameters techniques
1, Knowledge sharing
2, Transfer learning
2, Multi-task objectives and sharing
1, Towards generality
1, Efficiency and theory
2, Data Prioritization
2, Regularization
2, Bayesian Neural Nets
2, Theory about neural network e

In [ ]:
import re
from bs4 import NavigableString

all_strings = soup.body.find_all(["li","p", "h1","h2","h3","h4","h5","h6"])


# first and last item should be [category, last_linked]
ambiguous_items = list()

# [[category, next_category], [prev_linked, next_linked], [ambiguous_spans]]
ambiguous_all = list()

# [[category, first_link, text, all_links]]
parsed = list()
last_category = None
last_linked = None


for item in all_strings:
    # ignore categories
    item_string = item.text
    if item_string == "":
        continue
    if item_string in category_strings:
        # print "category", item_string, "total", len(parsed)
        last_category = item_string
        if ambiguous_items:
            ambiguous_items[1] = last_category
            ambiguous_all.append(ambiguous_items)
            ambiguous_items = list()
        continue
    links = list()
    for link in item.find_all("a"):
        if link.get("href"):
            links.append(link.get("href"))
    if links:
        last_linked = item_string
        if ambiguous_items:
            ambiguous_items[1] = last_category
            ambiguous_items[3] = last_linked
            ambiguous_all.append(ambiguous_items)
            ambiguous_items = list()
        addon = [last_category, item_string]
        addon.extend(links)
        parsed.append(addon)
    else:
        if not ambiguous_items:
            ambiguous_items = [last_category, None, last_linked, None]
        ambiguous_items.append(item_string)

print len(parsed)
print len(all_strings)

In [ ]:
# for item in parsed[100:120]:
#     print item[2]

# print len(ambiguous_all)

# for item in ambiguous_all:
#     print item[0]
#     print item[1]
#     if item[2]:
#         print "".join(list(item[2].strings))
#     else:
#         print item[2]
#     if item[3]:
#         print "".join(list(item[3].strings))
#     else:
#         print item[3]
#     for subitem in item[4:]:
#         print "----->",subitem
#     print "\n"

In [ ]:
import csv

# help from https://docs.python.org/2/library/csv.html

with open('doctable.csv', 'wb') as f:
    writer = csv.writer(f, delimiter="|")
    for row in parsed:
        writer.writerow([None if not s else s.encode("ascii", errors="xmlcharrefreplace") for s in row])

with open('ambiguous.csv', 'wb') as f:
    writer = csv.writer(f, delimiter="|")
    for row in ambiguous_all:
        writer.writerow([None if not s else s.encode("ascii", errors="xmlcharrefreplace") for s in row])